In [1]:
import pandas as pd
from recipe_scrapers import scrape_me
import time
import random
import json
import pymongo
from tqdm.notebook import tqdm
import numpy as np

In [2]:
def wait():
    
    x = random.randrange(50, 200, 1)/100
    print(f"Waiting for {x} Seconds.", end='\r')
    time.sleep(x)

In [3]:
# Db Information
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['food_analysis']
urls = db['urls']
recipes = db['recipes']

# Scraper
Take the url database and start retrieving the actual recipe contents to store in a new database dedicated to the recipes. Given that this database will form the basis for an actual app and analysis the key unique id will be generated by Mongo and keep close tabs with the url database to ensure no read recipes are reread.

In [ ]:
eligible_urls = [x["_id"] for x in list(urls.find({"read":False}, {"_id":1}))]
eligible_urls = sorted(eligible_urls, key = lambda x: random.random())

for url in tqdm(eligible_urls):
    
    # Obtain existing data
    print(url)
    row = list(urls.find({"_id":url}))[0]
    
    
    try:
        if url[0] != 'h':
            url = "https:" + url

        # Pause and then scrape
        wait()
        scraper = scrape_me(url)

        # Attempt each piece of data

        # Title
        try:
            title = scraper.title()
        except:
            try:
                title = row['name']
            except:
                title = None

        # Total Time
        try:
            total_time = scraper.total_time()
        except:
            total_time = None

        # Yields
        try:
            yields = scraper.yields()
        except:
            yields = None

        # Ingredients
        try:
            ingredients = scraper.ingredients()
        except:
            ingredients = []

        # Instructions
        try:
            instructions = scraper.instructions().encode('utf-8', 'surrogatepass')
        except:
            instructions = None

        # Image
        try:
            image = scraper.image()
        except:
            image = None

        # Ratings
        try:
            rating = scraper.ratings()
        except:
            rating = None

        # Author
        try:
            author = scraper.author()
        except:
            try:
                author = row['author']
            except:
                author = None

        # Reviews
        try:
            reviews = scraper.reviews()
        except:
            reviews = None

        # Insert new data
        recipes.insert_one({"title":title,
                            "total_time":total_time,
                            "yields":yields,
                            "ingredients":ingredients,
                            "instructions":instructions,
                            "image":image,
                            "rating":rating,
                            "author":author,
                            "reviews":reviews,
                            "source":row['source'],
                            "url":row["_id"]})

        # Mark as read
        query = {"_id":url}
        newvalues = {"$set":{"read":True}}
        urls.update_one(query, newvalues)
        
    except:
        
        # Mark as Error
        print("Error encountered.")
        query = {"_id":url}
        newvalues = {"$set":{"error":True}}
        urls.update_one(query, newvalues)

https://www.food.com/recipe/chicken-salad-for-sandwiches-96154
https://www.food.com/recipe/gerhard-jennes-gingerbread-men-59591
https://minimalistbaker.com/vegan-kahlua-hot-fudge/
https://www.epicurious.com/recipes/food/views/breakfast-grilled-cheese-with-soft-scrambled-eggs
https://www.epicurious.com/recipes/food/views/mocha-creme-caramel-1293
https://www.food.com/recipe/tava-fry-vegetables-264208
https://www.food.com/recipe/halloween-caramel-corn-44440
https://www.food.com/recipe/gluten-free-holiday-pumpkin-bread-199774
https://www.food.com/recipe/pineapple-raita-57437
https://www.southernliving.com/recipes/roasted-beet-citrus-salad-honey-orange-vinaigrette-recipe
https://www.food.com/recipe/chinese-chicken-lettuce-wraps-171913
https://minimalistbaker.com/1-pot-curried-chickpea-soup/
https://www.jamieoliver.com/recipes/fish-recipes/sloe-gin-gravlax/
https://www.epicurious.com/recipes/food/views/calabrian-grilled-pork-ribs-51264030
https://www.epicurious.com/recipes/food/views/roasted

https://www.food.com/recipe/cauliflower-dal-with-panch-phoron-352834
https://www.foodrepublic.com/recipes/turmeric-battered-cauliflower/
https://www.food.com/recipe/brazilian-coconut-cheese-drops-385024
https://www.food.com/recipe/calzone-with-sun-dried-tomatoes-189551
https://www.food.com/recipe/creamy-crock-pot-chicken-noodle-soup-507382
https://www.food.com/recipe/tropical-citrus-sauce-180369
https://www.epicurious.com/recipes/food/views/black-bean-lasagna-kinstlinger-bruhn-10253
https://www.foodnetwork.com/recipes/rachael-ray/milanese-fettuccini-alfredo-recipe-1949181
https://www.food.com/recipe/always-perfect-roast-turkey-507154
https://www.food.com/recipe/meat-lovers-chunky-mushroom-pasta-sauce-332161
https://www.southernliving.com/syndication/barbecue-peach-summer-rolls
https://www.food.com/recipe/chicken-tortilla-wrap-sandwich-126712
https://www.food.com/recipe/butternut-squash-and-parmesan-bread-pudding-262197
https://www.foodnetwork.com/recipes/melissa-darabian/pineapple-rasp

https://www.food.com/recipe/chocolate-crackle-cookies-146216
https://www.food.com/recipe/risotto-with-asparagus-mint-and-lemon-300438
https://www.food.com/recipe/lemonade-from-scratch-534764
https://www.food.com/recipe/bacon-and-balsamic-vinaigrette-137040
https://www.southernliving.com/recipes/grilled-wedge-salad-buttermilk-blue-cheese-dressing
https://www.food.com/recipe/lamb-stew-65313
https://www.food.com/recipe/indian-chicken-curry-448147
https://www.food.com/recipe/j-a-s-bbq-baked-beans-373971
https://www.epicurious.com/recipes/food/views/suffering-baptist-51184010
https://www.food.com/recipe/beef-stroganoff-306326
https://www.food.com/recipe/philadelphia-fruit-smoothie-no-bake-cheesecake-330396
https://www.mybakingaddiction.com/nutella-cheesecake-trifles/
https://www.food.com/recipe/christmas-wreath-cookies-443849
https://www.food.com/recipe/steel-cut-oatmeal-for-the-crock-pot-101101
https://www.food.com/recipe/baked-potato-soup-355095
https://cookieandkate.com/strawberry-oat-mu

https://www.food.com/recipe/apple-cake-42463
https://www.food.com/recipe/chicken-rice-casserole-132989
https://www.food.com/recipe/low-carb-breakfast-balls-98774
https://www.food.com/recipe/nees-angel-cornbread-183714
https://www.food.com/recipe/poached-lemon-garlic-chicken-294215
https://www.epicurious.com/recipes/food/views/sour-cream-and-lemon-pound-cake-2121
https://www.epicurious.com/recipes/food/views/key-lime-pie-230292
https://www.epicurious.com/recipes/food/views/poached-pineapple-in-cinnamon-syrup-with-yogurt-cheese-10405
https://www.food.com/recipe/soffrito-223296
https://www.food.com/recipe/apple-celery-salad-177783
https://www.cookstr.com/recipes/berry-shortcake
https://www.epicurious.com/recipes/food/views/peruvian-style-beef-kebabs-with-grilled-onion-and-zucchini-103661
https://www.food.com/recipe/bacon-ranch-dressing-16234
https://www.foodnetwork.com/recipes/sandra-lee/chile-corn-muffins-with-chipotle-butter-recipe-1953106
https://www.food.com/recipe/sea-breeze-scramble

https://www.food.com/recipe/roast-meat-and-gravy-262740
https://www.food.com/recipe/mexican-hot-corn-36063
https://www.food.com/recipe/thai-coconut-peanut-chicken-393293
https://www.food.com/recipe/pastel-de-choclo-243604
https://www.food.com/recipe/fat-free-tartar-sauce-324834
https://www.epicurious.com/recipes/food/views/chicken-with-black-bean-sauce-2878
https://www.food.com/recipe/feta-cheese-flavored-orzo-31245
https://www.food.com/recipe/chinese-hot-and-sour-soup-68544
https://www.foodnetwork.com/recipes/marcela-valladolid/spicy-sweet-potato-casserole-recipe-2119537
https://www.epicurious.com/recipes/food/views/smoked-salmon-and-basil-rolls-with-creme-fraiche-106612
https://www.closetcooking.com/greek-style-salmon-with-avocado-tzatziki/
https://www.food.com/recipe/vorarlberg-cheese-spaetzle-drop-noodles-148589
https://www.epicurious.com/recipes/food/views/vanilla-buttercream-51159620
https://www.food.com/recipe/fruity-pineapple-cake-14353
https://www.foodnetwork.com/recipes/racha

...

https://www.food.com/recipe/indonesian-fried-rice-ashmore-and-cartier-469937
https://www.cookstr.com/recipes/the-best-creamed-spinach
https://www.food.com/recipe/granola-blueberry-muffins-63263
https://www.foodnetwork.com/recipes/daisy-martinez/avocado-stuffed-with-crab-mango-salad-recipe-1948805
https://www.food.com/recipe/lenten-eggplant-aubergine-134038
https://www.food.com/recipe/white-chocolate-cranberry-biscotti-5136
https://www.epicurious.com/recipes/food/views/zeppole-363570
https://www.food.com/recipe/low-fat-cranberry-nut-bread-201705
https://www.epicurious.com/recipes/food/views/grilled-chicken-breasts-and-chipotle-cilantro-carrots-with-feta-358494
https://www.foodnetwork.com/recipes/tyler-florence/standing-rib-roast-with-cabernet-au-jus-recipe-1938903
https://www.food.com/recipe/eggnog-snowman-cookies-113300
https://www.food.com/recipe/organic-summer-pasta-salad-299250
https://www.food.com/recipe/potato-and-apple-salad-311979
https://www.epicurious.com/recipes/food/views/ho

https://www.food.com/recipe/grape-pudding-169156
https://www.closetcooking.com/burgers-smothered-in-caramelized-onion/
https://www.food.com/recipe/beef-barley-soup-452376
https://www.closetcooking.com/yorkshire-pudding/
https://www.cookstr.com/recipes/my-familyrsquos-favorite-hot-chocolate
https://www.food.com/recipe/thai-mango-with-sticky-rice-mamuang-kao-nieo-15584
https://www.food.com/recipe/anitas-chicken-and-dumplings-442149
https://www.food.com/recipe/thai-chicken-stew-with-potato-chive-dumplings-207388
https://www.epicurious.com/recipes/food/views/orange-scented-hot-chocolate-102810
https://www.southernliving.com/recipes/easy-pumpkin-chocolate-chip-cookies
https://www.food.com/recipe/juicy-pan-seared-pork-chops-with-citrus-dressing-537017
https://www.foodnetwork.com/recipes/claire-robinson/herb-buttered-noodles-recipe-1922671
https://www.food.com/recipe/fudgy-chocolate-layer-cake-40636
https://www.epicurious.com/recipes/food/views/northern-thai-chile-powder-109478
https://www.ep

https://www.food.com/recipe/quick-and-easy-banana-cream-pie-137055
https://www.food.com/recipe/vegetarian-black-bean-chili-68747
https://www.mybakingaddiction.com/rolo-cupcakes-recipe/
https://www.food.com/recipe/shrimp-mousse-on-sugar-cane-vietnamese-chao-tom-113940
https://www.food.com/recipe/creamy-cabbage-and-carrot-soup-171248
https://www.jamieoliver.com/recipes/bread-recipes/fried-pizza-pizza-fritta/
https://www.food.com/recipe/jalapeno-bacon-cheese-dip-538458
https://www.food.com/recipe/roasted-asparagus-mushroom-salad-with-creamy-mustard-dress-437149
https://www.food.com/recipe/healthy-corned-beef-hash-with-fried-eggs-222256
https://www.food.com/recipe/northern-jambalaya-310936
https://www.foodnetwork.com/recipes/robin-miller/picante-roast-beef-sandwich-with-garlic-lime-and-green-chile-recipe-1945876
https://www.food.com/recipe/mixed-berry-sauce-for-cheesecakes-and-desserts-527818
https://www.food.com/recipe/prawn-shrimp-satay-173814
https://www.food.com/recipe/grilled-mahi-mah

https://www.food.com/recipe/croatian-dalmatian-shrimp-s-chowder-326272
https://www.food.com/recipe/spiced-rice-and-lentils-with-cauliflower-447627
https://www.food.com/recipe/whalins-seven-grain-bread-449126
https://www.food.com/recipe/shirleys-corn-casserole-15862
https://www.food.com/recipe/cheesy-broccoli-soup-292199
https://www.food.com/recipe/ginger-lemon-muffins-199961
https://www.food.com/recipe/chicken-chili-151790
https://www.food.com/recipe/traditional-style-swiss-steak-12837
https://www.food.com/recipe/glazed-apricot-tart-with-frangipane-mini-tarts-tartlets-383037
https://www.closetcooking.com/chicken-and-mushroom-wild-rice-kale/
https://www.food.com/recipe/squaw-corn-casserole-72311
https://www.food.com/recipe/chicken-pot-pie-273330
https://www.food.com/recipe/wedding-cake-buttercream-frosting-double-batch-355821
https://www.food.com/recipe/seafood-pasta-salad-94610
https://www.food.com/recipe/enchiladas-verdes-lite-green-chicken-enchiladas-317689
https://www.food.com/recip

https://www.food.com/recipe/spiced-bat-wings-chicken-wings-192345
https://www.food.com/recipe/gingerbread-bread-pudding-76914
https://www.jamieoliver.com/recipes/vegetables-recipes/the-real-mushroom-soup/
https://www.food.com/recipe/sauteed-asparagus-and-snap-peas-149783
https://www.food.com/recipe/indonesian-fried-rice-nasi-goreng-61614
https://www.foodnetwork.com/recipes/sandra-lee/sauteed-zucchini-with-garlic-and-herbs-recipe-1937219
https://www.food.com/recipe/butter-rum-sandwich-cookies-72244
https://www.lecremedelacrumb.com/chocolate-chip-cookie-dough-hot-fudge-sundae/
https://www.food.com/recipe/baked-ranch-parmesan-chicken-96098
https://www.food.com/recipe/chocolate-molasses-raisin-cookies-334307
https://www.epicurious.com/recipes/food/views/fusilli-with-porcini-puttanesca-sauce-3178
https://www.food.com/recipe/ancho-chile-fries-185631
https://www.food.com/recipe/sweet-and-sour-sauce-philippine-style-149516
https://www.food.com/recipe/ww-2-pt-espresso-granita-358472
https://www

# Database Cleaning and Maintenance

In [4]:
# Print Overview
total = len(list(urls.find({})))
read = len(list(urls.find({"read":True})))
togo = len(list(urls.find({"read":False})))
errors = len(list(urls.find({"error":True})))
in_recipes = len(list(recipes.find({})))

print(f"URL Database: {total}")
print(f"Read: {read}")
print(f"To Read: {togo}")
print(f"Errors: {errors}")
print(f"Recipes Database: {in_recipes}")

URL Database: 226844
Read: 78853
To Read: 147991
Errors: 32
Recipes Database: 78853


### Duplicate Cleaning
At this stage should be true duplicates, if urls are found more than once it keeps first, which is arbitrary

In [53]:
# Remove Duplicates (true dups, everything is identical)
dups = pd.DataFrame(list(recipes.find({})))
dups = dups[["_id", "url"]].groupby("url", as_index=False).count()
dups = np.unique(dups[dups["_id"]>1]["url"])
for url in dups:
    delete = [x["_id"] for x in list(recipes.find({"url":url}))[1:]]
    for did in delete:
        recipes.delete_one({"_id":did})

### Marking Read
Depending on certai issues some documents don't get marked as read. If they are in recipes then they should be updated and this quick fix will clean that up

In [66]:
# Mark everything as read
read_urls = [x["url"] for x in list(recipes.find({}))]
for url in tqdm(read_urls):
    urls.update_one({"_id":url}, {"$set":{"read":True, "error":False}})